In [3]:
import numpy as np
import pickle
import sympy as sp
import multiprocessing as mp    
import os
import json
# print(os.getcwd())

def create_dir(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)
    else:
        print("Directory already existed : ", directory)
    return directory

Microelasticity modelling on mean square atomic displacement (MSAD) based on high entropy diborides (HEBs) lattice ($P6/mmm$; $11\bar{2}0-1\bar{1}00-0001$ coordinate system).

In [ ]:
def k_eval(lambda_k, kxyz):
    kx, ky, kz = kxyz 
    return np.abs(lambda_k)**2*np.exp(-a**2*kx**2-c**2*ky**2-3*a**2*kz**2)

specie_denote = '50Ti50ZrB2'
a, c = 3.11, 3.39
ti, v, zr = 0.5, 0, 0.5
nx, ny, nz = 32, 32, 32
eigenstrain_mode_list = ['a', 'c', 'a_sqrt3']
lattice_axis_list = ['1', '2', '3']

#* lattice vector and cluster volume
lattice_vec = np.array([
    [a, 0, 0],
    [0, c, 0],
    [0, 0, a*np.sqrt(3)]
])
lattice_vol = np.linalg.det(lattice_vec)
v_tmb2 = lattice_vol/2
lattice_vol_tot = lattice_vol*nx*ny*nz

msd_dict = {}
tot_msd = 0
xti, xv, xzr = sp.symbols('TiB_2 VB_2 ZrB_2')
for eigenstrain_mode, lattice_axis in zip(eigenstrain_mode_list, lattice_axis_list):

    #* load interaction kernel
    kernel_savpth = f'kernel/runs/{specie_denote}/{specie_denote}_{a}_{c}_{nx}_{ny}_{nz}'
    kmesh = np.load(f'{kernel_savpth}_kmesh.npy')
    prefac = np.load(f'{kernel_savpth}_lambda{lattice_axis}.npy')

    #* load eigenstrain
    eigen_express = pickle.load(open(f'eigenstrain_express/eigenstrain_{eigenstrain_mode}.pkl', 'rb'))
    eigen_ti, eigen_v, eigen_zr = eigen_express
    ti_eigen = eigen_ti.evalf(subs={xti:ti, xv:v, xzr:zr})
    v_eigen = eigen_v.evalf(subs={xti:ti, xv:v, xzr:zr})
    zr_eigen = eigen_zr.evalf(subs={xti:ti, xv:v, xzr:zr})

    var_eigenstrain = ti*ti_eigen**2 + v*v_eigen**2 + zr*zr_eigen**2

    #* evaluate the displacement's variance per axis
    p = mp.Pool(54)
    kernel_result = p.starmap(k_eval, zip(prefac, kmesh))

    msd = v_tmb2*var_eigenstrain/lattice_vol_tot*np.sum(kernel_result)*1e4 #* in pm^2
    msd_dict[f'{lattice_axis}_microEmodel'] = msd
    tot_msd += msd

sav_pth = f'runs/{specie_denote}/{specie_denote}_{a}_{c}_{nx}_{ny}_{nz}_msad.pkl'
create_dir(os.path.dirname(sav_pth))
pickle.dump(msd_dict, open(sav_pth, 'wb'))

msd_dict, tot_msd

({'1_microEmodel': 3.10809145995341,
  '2_microEmodel': 18.5846509325613,
  '3_microEmodel': 3.18063587056644},
 24.8733782630811)